In [3]:
import json
import os
import sys

import boto3
import botocore

In [4]:
bedrock = boto3.client("bedrock-runtime")

In [5]:
prompt_data = """ Human: 
Sulfuric acid reacts with sodium chloride, and gives <chemical1>_____</chemical1> and <chemical2>_____</chemical2>:
Assistant: the chemical1 and chemical 2 are:
"""

In [6]:
body = json.dumps(
    {
        "inputText": prompt_data,
        "textGenerationConfig": {"maxTokenCount": 1024, "topP": 0.95, "temperature": 0.1},
    }
)

In [16]:
output = []
modelId = "amazon.titan-text-express-v1"  # "amazon.titan-tg1-large"
accept = "application/json"
contentType = "application/json"
outputText = "\n"
try:

    response = bedrock.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    stream = response.get("body")

    i = 1
    if stream:
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                chunk_obj = json.loads(chunk.get("bytes").decode())
                text = chunk_obj["outputText"]
                output.append(text)
                print(f"\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n")
                i += 1

except botocore.exceptions.ClientError as error:

    if error.response["Error"]["Code"] == "AccessDeniedException":
        print(
            f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n"
        )

    else:
        raise error

		**Chunk 1**
Sulfuric acid reacts with sodium chloride, and gives <chemical1>Sodium sulfate</chemical1> and <chemical2>Hydrochloric acid</chemical2>:

